In [2]:
import numpy as np
import pandas as pd
from astropy.io import fits
import plotly.graph_objects as go
import plotly.express as px # type: ignore

import dustmaps
from dustmaps.edenhofer2023 import Edenhofer2023Query

# Querey dustmap

In [4]:
# Only run if edehofer2023 is not already downloaded
# import dustmaps.edenhofer2023
# dustmaps.edenhofer2023.fetch()

# Querey the stellar cluster data

# Read in Data